In [1]:
#import packages
import numpy as np
import json 
import scanpy as sc
from collections import OrderedDict
import scipy 
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
#spectra imports 
import Spectra as spc
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets

In [2]:
with open("/data/zhouweiwei/DL/BERT-pytorch-master/OUT/spectra-main/build/lib/Spectra/data/default_gene_sets.json", 'r') as f:
     default_gene_sets = json.loads(f.read())
annotations = default_gene_sets
annotations["global"]["all_wnt-beta-catenin-signaling"]
obs_key = 'cell_type_annotations'
adata = adata = sc.read_h5ad("/data/zhouweiwei/DL/BERT-pytorch-master/OUT/spectra-main/build/lib/Spectra/data/sample_data.h5ad")
adata

AnnData object with n_obs × n_vars = 1000 × 6397
    obs: 'cell_type_annotations'
    var: 'n_cells_by_counts', 'highly_variable'
    uns: 'annotation_SPADE_1_colors', 'cell_type_annotations_colors', 'diffmap_evals', 'draw_graph', 'hvg', 'neighbors', 'pca'
    obsm: 'X_diffmap', 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [3]:
list(set(adata.obs[obs_key]))

['ILC3',
 'B_naive',
 'CD4_T',
 'B_GC',
 'DC',
 'B_memory',
 'gdT',
 'Treg',
 'MDC',
 'mast',
 'NK',
 'CD8_T',
 'pDC',
 'plasma']

In [4]:
list(set(annotations.keys()))

['B_naive',
 'CD4_T',
 'DC',
 'B_memory',
 'gdT',
 'Treg',
 'global',
 'MDC',
 'mast',
 'plasma',
 'B_GC',
 'NK',
 'CD8_T',
 'pDC',
 'ILC3']

In [5]:
annotations = spc_tl.check_gene_set_dictionary(
    adata,
    annotations,
    obs_key='cell_type_annotations',
    global_key='global')

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


In [6]:
model = spc.est_spectra(adata=adata, 
    gene_set_dictionary=annotations, 
    use_highly_variable=True,
    cell_type_key="cell_type_annotations", 
    use_weights=True,
    lam=0.1, # varies depending on data and gene sets, try between 0.5 and 0.001
    delta=0.001, 
    kappa=None,
    rho=0.001, 
    use_cell_types=True,
    n_top_vals=50,
    label_factors=True, 
    overlap_threshold=0.2,
    clean_gs = True, 
    min_gs_num = 3,
    num_epochs=2 #here running only 2 epochs for time reasons, we recommend 10,000 epochs for most datasets
)


Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.89s/it]
/data/zhouweiwei/0-soft/miconda/envs/D2L/lib/python3.9/site-packages/scSpectra-0.1.0-py3.9.egg/Spectra/Spectra.py:1340: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'BTD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
/data/zhouweiwei/0-soft/miconda/envs/D2L/lib/python3.9/site-packages/scSpectra-0.1.0-py3.9.egg/Spectra/Spectra.py:1340: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SLC5A6' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
/data/zhouweiwei/0-soft/miconda/envs/D2L/lib/python3.9/site-packages/scSpectra-0.1.0-py3.9.egg/Spectra/Spectra.py:1340: FutureWa

In [7]:
factors = adata.uns['SPECTRA_factors'] # factors x genes matrix that tells you how important each gene is to the resulting factors
markers = adata.uns['SPECTRA_markers'] # factors x n_top_vals list of n_top_vals top markers per factor
cell_scores = adata.obsm['SPECTRA_cell_scores'] # cells x factors matrix of cell scores
vocab = adata.var['spectra_vocab'] # boolean matrix of size # of genes that indicates the set of genes used to fit spectra 

In [10]:
from Spectra import K_est as kst
L = kst.estimate_L(adata, attribute = "cell_type_annotations", highly_variable = True)


Step 1; estimating best fit null distribution... 
Minimization gap...  5.9
Minimization gap...  3.933333333333333
Minimization gap...  2.6222222222222222
Minimization gap...  1.7481481481481482
Minimization gap...  1.165432098765432
Minimization gap...  0.776954732510288
Minimization gap...  0.5179698216735253
Minimization gap...  0.34531321444901686
Minimization gap...  0.23020880963267787
Minimization gap...  0.15347253975511854
Minimization gap...  0.10231502650341234
Minimization gap...  0.06821001766894158
Minimization gap...  0.04547334511262774
Minimization gap...  0.03031556340841851
Minimization gap...  0.02021037560561234
Minimization gap...  0.013473583737074912
Minimization gap...  0.004491194579024971
Minimization gap...  0.0029941297193499805
Step 2; estimating K...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:41<00:00,  2.42it/s]


Step 1; estimating best fit null distribution... 
Minimization gap...  5.9
Minimization gap...  1.9666666666666668
Minimization gap...  0.6555555555555556
Minimization gap...  0.21851851851851853
Minimization gap...  0.07283950617283949
Minimization gap...  0.024279835390946497
Minimization gap...  0.008093278463648837
Minimization gap...  0.005395518975765887
Minimization gap...  0.0017985063252553002
Minimization gap...  0.0011990042168368714
Step 2; estimating K...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 723.73it/s]


Step 1; estimating best fit null distribution... 
Minimization gap...  5.9
Minimization gap...  1.9666666666666668
Minimization gap...  0.6555555555555556
Minimization gap...  0.21851851851851853
Minimization gap...  0.07283950617283949
Minimization gap...  0.048559670781892994
Minimization gap...  0.01618655692729766
Minimization gap...  0.010791037951531773
Minimization gap...  0.007194025301021187
Minimization gap...  0.002398008433673729
Minimization gap...  0.001598672289115824
Minimization gap...  0.0010657815260772113
Step 2; estimating K...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 582.16it/s]


Step 1; estimating best fit null distribution... 
Minimization gap...  5.9
Minimization gap...  1.9666666666666668
Minimization gap...  0.6555555555555556
Minimization gap...  0.21851851851851853
Minimization gap...  0.07283950617283949
Minimization gap...  0.048559670781892994
Minimization gap...  0.01618655692729766
Minimization gap...  0.005395518975765887
Minimization gap...  0.0017985063252553002
Minimization gap...  0.0011990042168368714
Step 2; estimating K...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 584.11it/s]


Step 1; estimating best fit null distribution... 
Minimization gap...  5.9
Minimization gap...  1.9666666666666668
Minimization gap...  0.6555555555555556
Minimization gap...  0.21851851851851853
Minimization gap...  0.07283950617283949
Minimization gap...  0.024279835390946497
Minimization gap...  0.01618655692729766
Minimization gap...  0.005395518975765887
Minimization gap...  0.0017985063252553002
Step 2; estimating K...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 235.01it/s]


In [12]:
adata.uns['SPECTRA_overlap']

,B_Breg_UP,B_effector-2_UP,B_effector-1_UP,B_IgM-ligation_response,CD4-T_IL12_response,TNK_IL2_response,CD4-T_TH22_UP,CD4-T_TH17_UP,TNK_IL2-STAT5-signaling,T_IL21_response,...,all_thiamin_metabolism,all_NOTCH_signaling,all_galactose_metabolism,all_MYC_targets,all_GLU_metabolism,all_fatty-acid_synthesis,all_glycolysis,all_glutathione_metabolism,all_IL6-JAK-STAT3_signaling,all_autophagy-chaperone-mediated
0-X-global-X-all_biotin_metabolism,0.0,0.0,0.0,0.02,0.000000,0.000000,0.0,0.0,0.02,0.0,...,0.0,0.000000,0.000000,0.08,0.000000,0.000000,0.000000,0.0,0.00,0.0
1-X-global-X-all_purine_metabolism,0.2,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.04,0.0,...,0.0,0.000000,0.000000,0.02,0.000000,0.000000,0.000000,0.0,0.00,0.0
2-X-global-X-all_ethanol_metabolism,0.0,0.2,0.0,0.04,0.076923,0.000000,0.0,0.0,0.04,0.0,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.041667,0.0,0.00,0.0
3-X-global-X-all_amino-sugar-nucleotide-sugar_metabolism,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.04,0.0,...,0.0,0.029412,0.055556,0.02,0.000000,0.000000,0.041667,0.0,0.00,0.0
4-X-global-X-all_steroid_metabolism,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191-X-mast-X-mast_granule-exocytosis,0.0,0.0,0.0,0.00,0.076923,0.000000,0.0,0.0,0.10,0.0,...,0.0,0.029412,0.000000,0.00,0.027027,0.000000,0.000000,0.0,0.04,0.0
192-X-mast-X-192,0.0,0.0,0.0,0.04,0.000000,0.055556,0.0,0.0,0.00,0.0,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.02,0.0
193-X-pDC-X-p-DC_CpG-TLR9_response,0.0,0.0,0.0,0.02,0.000000,0.055556,0.0,0.0,0.02,0.0,...,0.0,0.000000,0.000000,0.02,0.027027,0.000000,0.000000,0.0,0.00,0.0
194-X-pDC-X-194,0.0,0.0,0.0,0.02,0.000000,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.029412,0.000000,0.00,0.027027,0.000000,0.000000,0.0,0.02,0.0
